In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# 根據Mask的輪廓獲取旋轉角度的函數
def get_rotation_angle(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return 0
    cnt = contours[0]
    rect = cv2.minAreaRect(cnt)
    angle = rect[2]
    if rect[1][0] > rect[1][1]:
        angle += 90
    if angle > 90:
        angle += 180
    return angle

# 按指定角度旋轉圖像的函數
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

# 在旋轉後將座標轉回原始位置的函數
def convert_coord_back(coord, angle, image):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    M_inv = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    coord_ones = np.array([coord[0], coord[1], 1.0])
    original_coord_back = M_inv.dot(coord_ones)
    original_coord_back = original_coord_back[:2].astype(int)
    return original_coord_back

# 檢查一個值是否在某個範圍內的函數
def is_within_range(value, target, range_size=50):
    return target - range_size <= value <= target + range_size

# 分配非 None 值給變量的函數
def assign_non_none_values(dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y):
    if dentin_left_x is None and dentin_right_x is not None:
        dentin_left_x = dentin_right_x
    elif dentin_right_x is None and dentin_left_x is not None:
        dentin_right_x = dentin_left_x
    if dentin_left_y is None and dentin_right_y is not None:
        dentin_left_y = dentin_right_y
    elif dentin_right_y is None and dentin_left_y is not None:
        dentin_right_y = dentin_left_y
    return dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y

# 根據 y 坐標從輪廓中獲取前 100 點的函數
def get_top_100_points(contours, reverse=True):
    all_points = []
    for contour in contours:
        sorted_points = sorted(contour, key=lambda x: x[0][1], reverse=reverse)
        top_points = sorted_points
        all_points.extend(top_points)
    all_points = sorted(all_points, key=lambda x: x[0][1], reverse=reverse)
    return all_points

# 計算兩個點之間距離的函數
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# 根據距離值 AB 和 AC 計算百分比和階段的函數
def calculate_percentage_and_stage(AB, AC):
    if AC == 0:
        return 0, "1"
    
    percentage = math.floor((AB / AC) * 100)  # 無條件捨去小數點
    if percentage < 15:
        stage = "1"
    elif 15 <= percentage <= 33:
        stage = "2"
    else:
        stage = "3"
    
    return percentage, stage

# 並排顯示帶有註釋的圖片的函數
def display_images(image, line_image, title="Image with Annotations"):
    fig, axes = plt.subplots(1, 2, figsize=(15, 15))
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"{title} - Annotated")
    axes[0].axis('off')
    axes[1].imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"{title} - Lines")
    axes[1].axis('off')
    plt.show()

# 處理圖像的主函數
def main():
    dir_input = 'C:/teeth1/150'
    output_path = 'C:/teeth1/result5'
    
    dir_list = os.listdir(dir_input)
    
    for target_dir in dir_list:
        dir_path = os.path.join(dir_input, target_dir)
        if not os.path.isdir(dir_path):
            continue

        # 初始化座標 DataFrame
        coord_df = pd.DataFrame(columns=['Tooth_ID', 'Tooth_Tip_ID', 'Mid_X', 'Mid_Y', 'Enamel_X', 'Enamel_Y',
                                         'Gum_X', 'Gum_Y', 'Dentin_X', 'Dentin_Y', 'AB', 'AC', 'Percentage', 'stage'])
        
        # 初始化 tooth_tip_id 
        tooth_tip_id = 1
        
        # 定義Mask圖片和原始圖片的path
        gum_mask_path = os.path.join(dir_path, f"gum_{target_dir}.png")
        teeth_mask_path = os.path.join(dir_path, f"teeth_{target_dir}.png")
        dental_crown_path = os.path.join(dir_path, f"DentalCrown_{target_dir}.png")
        dentin_path = os.path.join(dir_path, f"dentin_{target_dir}.png")
        original_img_path = os.path.join(dir_path, f"raw_{target_dir}.png")
        print(f"Processing {original_img_path} ...")

         # 載入圖片
        gum_img = cv2.imread(gum_mask_path, cv2.IMREAD_GRAYSCALE)
        teeth_img = cv2.imread(teeth_mask_path, cv2.IMREAD_GRAYSCALE)
        dental_crown_img = cv2.imread(dental_crown_path, cv2.IMREAD_GRAYSCALE)
        dentin_img = cv2.imread(dentin_path, cv2.IMREAD_GRAYSCALE)
        original_img = cv2.imread(original_img_path)

        # 將圖片二值化以創建二值化的Mask
        _, gum_bin = cv2.threshold(gum_img, 128, 255, cv2.THRESH_BINARY)
        _, teeth_bin = cv2.threshold(teeth_img, 128, 255, cv2.THRESH_BINARY)
        _, dental_crown_bin = cv2.threshold(dental_crown_img, 128, 255, cv2.THRESH_BINARY)
        _, dentin_bin = cv2.threshold(dentin_img, 128, 255, cv2.THRESH_BINARY)

        # 在Mask的圖片中找到輪廓
        contours_gum, _ = cv2.findContours(gum_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_teeth, _ = cv2.findContours(teeth_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_dental_crown, _ = cv2.findContours(dental_crown_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_dentin, _ = cv2.findContours(dentin_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

         # 複製原始圖片以進行標註
        image = original_img.copy()
        line_image = image.copy()

         # 定義形態學操作的內核
        kernel = np.ones((3, 3), np.uint8)

         # 獲取圖像尺寸
        height, width = dental_crown_bin.shape

        # 為未遮罩的區域創建遮罩
        non_masked_area = np.ones((height, width), dtype=np.uint8) * 255

        # 對牙冠Mask執行形態學操作
        dental_crown_bin = cv2.erode(dental_crown_bin, kernel, iterations=5)
        dental_crown_bin = cv2.dilate(dental_crown_bin, kernel, iterations=5)

        # 在牙冠Mask中找到連接的Components
        num_labels, labels = cv2.connectedComponents(dental_crown_bin)
        label_counts = np.bincount(labels.flatten())
        pixel_threshold = 2000
        filtered_image = np.zeros_like(gum_bin)

        # 過濾掉小的Components
        for label in range(1, num_labels):
            if label_counts[label] > pixel_threshold:
                filtered_image[labels == label] = 255
        dental_crown_bin = filtered_image

         # 對牙本質Mask執行形態學操作
        dentin_bin = cv2.erode(dentin_bin, kernel, iterations=1)
        dentin_bin = cv2.dilate(dentin_bin, kernel, iterations=1)

         # 在牙齦Mask中找到連接的Components
        num_labels, labels = cv2.connectedComponents(gum_bin)
        label_counts = np.bincount(labels.flatten())
        max_label = np.argmax(label_counts[1:]) + 1
        largest_component = np.zeros_like(gum_bin)
        largest_component[labels == max_label] = 255
        gum_bin = largest_component

        # 放大並侵蝕牙齦Mask
        dilated_gum_bin = cv2.dilate(gum_bin, kernel, iterations=10)
        gum_bin = cv2.erode(gum_bin, kernel, iterations=5)
        gum_bin = cv2.dilate(gum_bin, kernel, iterations=5)

        # 在牙齦Mask中再次找到連接的Components
        num_labels, labels = cv2.connectedComponents(gum_bin)
        label_counts = np.bincount(labels.flatten())
        max_label = np.argmax(label_counts[1:]) + 1
        largest_component = np.zeros_like(gum_bin)
        largest_component[labels == max_label] = 255
        gum_bin = largest_component

        # 合併牙齒不同區域的Mask
        combined_mask = cv2.bitwise_or(dilated_gum_bin, teeth_bin)
        combined_mask = cv2.bitwise_or(combined_mask, dental_crown_bin)
        combined_mask = cv2.bitwise_or(combined_mask, dentin_bin)

        # 為未遮罩的區域創建遮罩
        non_masked_area = cv2.bitwise_not(combined_mask)
        dilated_non_masked_area = cv2.dilate(non_masked_area, kernel, iterations=10)

        # 在牙本質Mask中找到輪廓
        num_labels, labels = cv2.connectedComponents(dentin_bin)

        # 在原始圖片上疊加Mask顏色
        overlay = image.copy()
        overlay[dental_crown_bin > 0] = (163, 118, 158)
        overlay[dentin_bin > 0] = (117, 122, 152)
        overlay[gum_bin > 0] = (0, 177, 177)

        # 循環處理牙本質Mask中的連接Components
        for i in range(1, num_labels):
            component_mask = np.uint8(labels == i) * 255
            area = cv2.countNonZero(component_mask)
            if area < 500:
                continue

            contours, _ = cv2.findContours(component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rect = cv2.minAreaRect(contours[0])
            box = cv2.boxPoints(rect)
            box = np.array(box, dtype=int)
            width = rect[1][0]
            height = rect[1][1]
            short_side = min(width, height)
            long_side = max(width, height)
            if short_side < 120:
                continue

            # 獲取旋轉角度並旋轉圖片和Mask
            angle = get_rotation_angle(component_mask)
            if is_within_range(short_side, long_side, 30):
                angle = 0
            c_component_mask = rotate_image(component_mask, angle)
            c_image = rotate_image(image, angle)

            # 擴大Mask並找到Components的中點
            dilated_mask = cv2.dilate(component_mask, kernel, iterations=7)
            non_zero_points = np.column_stack(np.where(dilated_mask > 0))
            if len(non_zero_points) > 0:
                mid_point = np.mean(non_zero_points, axis=0).astype(int)
                mid_y, mid_x = mid_point
                cv2.putText(image, str(i), (mid_x-5, mid_y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 1, cv2.LINE_AA)
                cv2.circle(image, (mid_x, mid_y), 5, (255, 255, 0), -1)

            # 找到與牙冠和牙齦Mask的交集
            intersection = cv2.bitwise_and(dental_crown_bin, dilated_mask)
            intersection_with_non_masked = cv2.bitwise_and(dilated_non_masked_area, intersection)
            overlay[intersection > 0] = (255, 0, 0)
            contours, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) == 0:
                continue

            # 獲取輪廓中的前 100 點並找到牙釉質座標
            top_points = get_top_100_points(contours, reverse=True)
            corners = top_points
            if corners is not None:
                corners = np.array(corners, dtype=int)
                enamel_left_x = None
                enamel_left_y = None
                enamel_right_x = None
                enamel_right_y = None
                for corner in corners:
                    x, y = corner.ravel()
                    if x < mid_x:
                        if enamel_left_x is not None:
                            if y > enamel_left_y:
                                enamel_left_x = x
                                enamel_left_y = y
                                continue
                            elif is_within_range(x, enamel_left_x):
                                continue
                        else:
                            enamel_left_x = x
                            enamel_left_y = y
                    elif x > mid_x:
                        if enamel_right_x is not None:
                            if y > enamel_right_y:
                                enamel_right_x = x
                                enamel_right_y = y
                                continue
                            elif is_within_range(x, enamel_right_x):
                                continue
                        else:
                            enamel_right_x = x
                            enamel_right_y = y

            # 找到與牙齦Mask的交集                
            intersection = cv2.bitwise_and(gum_bin, dilated_mask)
            intersection_with_non_masked = cv2.bitwise_and(dilated_non_masked_area, intersection)
            overlay[intersection > 0] = (0, 255, 0)
            contours, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) == 0:
                continue

            # 獲取輪廓中的前 100 點並找到牙齦座標
            top_points = get_top_100_points(contours, reverse=False)
            corners = top_points
            if corners is not None:
                corners = np.array(corners, dtype=int)
                top_corners = corners
                gum_left_x = None
                gum_left_y = None
                gum_right_x = None
                gum_right_y = None
                for corner in top_corners:
                    x, y = corner.ravel()
                    if x >= mid_x - 40 and x <= mid_x + 40:
                        continue
                    if x < mid_x:
                        if gum_left_x is not None:
                            if y < gum_left_y:
                                gum_left_x = x
                                gum_left_y = y
                                continue
                            elif is_within_range(x, gum_left_x):
                                continue
                        else:
                            gum_left_x = x
                            gum_left_y = y
                    elif x > mid_x:
                        if gum_right_x is not None:
                            if y < gum_right_y:
                                gum_right_x = x
                                gum_right_y = y
                                continue
                            elif is_within_range(x, gum_right_x):
                                continue
                        else:
                            gum_right_x = x
                            gum_right_y = y

             # 旋轉回圖片和座標
            angle = get_rotation_angle(component_mask)
            if is_within_range(short_side, long_side, 30):
                angle = 0
            c_component_mask = rotate_image(component_mask, angle)
            c_image = rotate_image(image, angle)
            c_intersection = rotate_image(intersection, angle)
            c_dilated_mask = rotate_image(dilated_mask, angle)
            non_zero_points = np.column_stack(np.where(c_dilated_mask > 0))
            if len(non_zero_points) > 0:
                mid_point = np.mean(non_zero_points, axis=0).astype(int)
                c_mid_y, c_mid_x = mid_point

            # 不同處理大的Components   
            if short_side > 270:
                contours, _ = cv2.findContours(c_intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) == 0:
                    continue
                top_points = get_top_100_points(contours, reverse=True)
                bottom_corners = top_points
                left_corners = []
                right_corners = []
                for point in bottom_corners:
                    x, y = point.ravel()
                    if x < c_mid_x - 80:
                        left_corners.append(point)
                    elif x >= c_mid_x - 80 and x <= c_mid_x:
                        continue
                    else:
                        right_corners.append(point)
                dentin_left_x = None
                dentin_left_y = None
                dentin_right_x = None
                dentin_right_y = None
                for corner in left_corners:
                    x, y = corner.ravel()
                    if x < c_mid_x:
                        if dentin_left_x is not None:
                            if y > dentin_left_y:
                                dentin_left_x = x
                                dentin_left_y = y
                                continue
                        else:
                            dentin_left_x = x
                            dentin_left_y = y
                for corner in right_corners:
                    x, y = corner.ravel()
                    if x > c_mid_x:
                        if dentin_right_x is not None:
                            if y > dentin_right_y:
                                dentin_right_x = x
                                dentin_right_y = y
                                continue
                        else:
                            dentin_right_x = x
                            dentin_right_y = y

                # 如果其中一個座標為 None 則分配值
                dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y = assign_non_none_values(dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y)
                if dentin_left_x is None:
                    continue
                else:
                    cv2.circle(c_image, (dentin_left_x, dentin_left_y), 5, (255, 0, 0), -1)
                    cv2.circle(c_image, (dentin_right_x, dentin_right_y), 5, (255, 0, 0), -1)
            else:
                contours, _ = cv2.findContours(c_intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if len(contours) == 0:
                    continue
                top_points = get_top_100_points(contours, reverse=True)
                bottom_corners = top_points
                dentin_left_x = None
                dentin_left_y = None
                dentin_right_x = None
                dentin_right_y = None
                for corner in bottom_corners:
                    x, y = corner.ravel()
                    if x < c_mid_x:
                        if dentin_left_x is not None:
                            if y > dentin_left_y:
                                dentin_left_x = x
                                dentin_left_y = y
                                continue
                        else:
                            dentin_left_x = x
                            dentin_left_y = y
                    elif x > c_mid_x:
                        if dentin_right_x is not None:
                            if y > dentin_right_y:
                                dentin_right_x = x
                                dentin_right_y = y
                                continue
                        else:
                            dentin_right_x = x
                            dentin_right_y = y
                # 如果其中一個座標為 None 則分配值
                dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y = assign_non_none_values(dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y)
                if dentin_left_x is None:
                    continue
                elif is_within_range(dentin_left_x, dentin_right_x, 80):
                    bottom_corner = bottom_corners[:1]
                    for corner in bottom_corner:
                        x, y = corner.ravel()
                        dentin_left_x = x
                        dentin_left_y = y
                        dentin_right_x = x
                        dentin_right_y = y
                        cv2.circle(c_image, (x, y), 5, (255, 0, 0), -1)
                else:
                    cv2.circle(c_image, (dentin_left_x, dentin_left_y), 5, (255, 0, 0), -1)
                    cv2.circle(c_image, (dentin_right_x, dentin_right_y), 5, (255, 0, 0), -1)
            
            # 將座標轉回原始圖片
            dentin_left_coord = [dentin_left_x, dentin_left_y]
            dentin_right_coord = [dentin_right_x, dentin_right_y]
            dentin_left_x, dentin_left_y = convert_coord_back(dentin_left_coord, angle, component_mask)
            dentin_right_x, dentin_right_y = convert_coord_back(dentin_right_coord, angle, component_mask)
            c_dilated_mask = cv2.dilate(c_dilated_mask, kernel, iterations=5)
            c_dilated_mask_rotated_back = rotate_image(c_dilated_mask, -angle)
            c_image_rotated_back = rotate_image(c_image, -angle)
            mask = c_dilated_mask_rotated_back > 0
            image[mask] = c_image_rotated_back[mask]

            # 計算AB和AC的距離
            if enamel_left_x is not None and gum_left_x is not None and dentin_left_x is not None:
                AB_left = calculate_distance(enamel_left_x, enamel_left_y, gum_left_x, gum_left_y)
                AC_left = calculate_distance(enamel_left_x, enamel_left_y, dentin_left_x, dentin_left_y)
                percentage_left, stage_left = calculate_percentage_and_stage(AB_left, AC_left)
            else:
                AB_left = AC_left = percentage_left = stage_left = None

            if enamel_right_x is not None and gum_right_x is not None and dentin_right_x is not None:
                AB_right = calculate_distance(enamel_right_x, enamel_right_y, gum_right_x, gum_right_y)
                AC_right = calculate_distance(enamel_right_x, enamel_right_y, dentin_right_x, dentin_right_y)
                percentage_right, stage_right = calculate_percentage_and_stage(AB_right, AC_right)
            else:
                AB_right = AC_right = percentage_right = stage_right = None

            # 新增左側的資料
            new_row_left = pd.DataFrame({
                'Tooth_ID': [i],
                'Tooth_Tip_ID': [tooth_tip_id],  # 使用遞增的 Tooth_Tip_ID
                'Mid_X': [mid_x], 'Mid_Y': [mid_y],
                'Enamel_X': [enamel_left_x], 'Enamel_Y': [enamel_left_y],
                'Gum_X': [gum_left_x], 'Gum_Y': [gum_left_y],
                'Dentin_X': [dentin_left_x], 'Dentin_Y': [dentin_left_y],
                'AB': [AB_left], 'AC': [AC_left],
                'Percentage': [percentage_left], 'stage': [stage_left]
            })
            coord_df = pd.concat([coord_df, new_row_left], ignore_index=True)
            tooth_tip_id += 1  # update更新計數器

            # 新增右側的資料
            new_row_right = pd.DataFrame({
                'Tooth_ID': [i],
                'Tooth_Tip_ID': [tooth_tip_id],  # 使用遞增的 Tooth_Tip_ID
                'Mid_X': [mid_x], 'Mid_Y': [mid_y],
                'Enamel_X': [enamel_right_x], 'Enamel_Y': [enamel_right_y],
                'Gum_X': [gum_right_x], 'Gum_Y': [gum_right_y],
                'Dentin_X': [dentin_right_x], 'Dentin_Y': [dentin_right_y],
                'AB': [AB_right], 'AC': [AC_right],
                'Percentage': [percentage_right], 'stage': [stage_right]
            })
            coord_df = pd.concat([coord_df, new_row_right], ignore_index=True)
            tooth_tip_id += 1  # update更新計數器

            # 繪製標記
            cv2.circle(image, (enamel_left_x, enamel_left_y), 5, (0, 0, 255), -1)
            cv2.circle(image, (enamel_right_x, enamel_right_y), 5, (0, 0, 255), -1)
            cv2.circle(image, (gum_left_x, gum_left_y), 5, (0, 255, 0), -1)
            cv2.circle(image, (gum_right_x, gum_right_y), 5, (0, 255, 0), -1)
            if enamel_left_x is not None:
                cv2.line(line_image, (enamel_left_x, enamel_left_y), (dentin_left_x, dentin_left_y), (0, 0, 255), 2)
            if (enamel_left_x is not None) and (gum_left_x is not None):
                cv2.line(line_image, (enamel_left_x, enamel_left_y), (gum_left_x, gum_left_y), (0, 255, 255), 2)
            if enamel_right_x is not None:
                cv2.line(line_image, (enamel_right_x, enamel_right_y), (dentin_right_x, dentin_right_y), (0, 0, 255), 2)
            if (enamel_right_x is not None) and (gum_right_x is not None):
                cv2.line(line_image, (enamel_right_x, enamel_right_y), (gum_right_x, gum_right_y), (0, 255, 255), 2)
        
        # 將疊加層與原始圖片混合
        cv2.addWeighted(overlay, 0.3, image, 0.7, 0, image)

        # 顯示圖片
        display_images(image, line_image, title=f"Output for {target_dir}")

         # 保存輸出圖片
        cv2.imwrite(os.path.join(dir_path, f'output_{target_dir}.png'), image)
        cv2.imwrite(os.path.join(dir_path, f'output_line_{target_dir}.png'), line_image)
        
        # 將每個目標的座標資料寫入 excel文件
        excel_filename = f"{target_dir}_coordinates.xlsx"
        coord_df.to_excel(os.path.join(output_path, excel_filename), index=False)
         
        # 重新讀取 Excel 文件，移除指定欄位
        coord_df = pd.read_excel(os.path.join(output_path, excel_filename))
        coord_df = coord_df.drop(columns=['Mid_X', 'Mid_Y', 'AB', 'AC'])
        
        # 再次保存回 excel檔案
        coord_df.to_excel(os.path.join(output_path, excel_filename), index=False)
 


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "E:\ProgramData\Anaconda3\envs\tf2-6_cv\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "E:\ProgramData\Anaconda3\envs\tf2-6_cv\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "E:\ProgramData\Anaconda3\envs\tf2-6_cv\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "E:\ProgramData\Anaconda3\envs\tf2-6_cv\lib\site-packages\IPython\core\ultratb.py", line 1030, in structured_traceback
    formatted_exception = self.format_exception_as_a_whole(etype, evalue, etb, number_of_lines_of_context,
  File "E:\ProgramData\Anaconda3\envs\tf2-6_cv\lib\site-packages\IPython\core\ultratb.py", line 935, in format_exception_as_a_whole
    self.get_records(etb, number_of_lines_of_context, tb_offset) if etb else []
 

ImportError: numpy.core.multiarray failed to import